In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True

In [ ]:
import sys
sys.path.append("..")

from ggqpy import construct_Chebyshev_quadratures
from ggqpy.functionfamiliy import Interval, FunctionFamily
from ggqpy.testproblems import example_problem, gen_poly_and_sing
from ggqpy.discretize import Discretizer
from ggqpy.compress import compress_sequence_of_functions, visualise_diagonal_dropoff, construct_A_matrix, interp_legendre
from ggqpy.optimize import QuadOptimizer
from ggqpy.visualize import plot_points

In [ ]:
gamma = (
    lambda r0, theta0, u: r0
    * np.sin(theta0)
    / (r0 * np.sin(theta0 - theta0 * u) + np.sin(theta0 * u))
)
number_of_parameters = 16

x_gl, _ = np.polynomial.legendre.leggauss(number_of_parameters)

(amin, amax) = (1e-7, 1)
alphas = (amax - amin) * (x_gl + 1) / 2 + amin

(bmin, bmax) = (1e-7, np.pi)
betas = (bmax - bmin) * (x_gl + 1) / 2 + bmin

n = 2
functions_cos = [
    lambda u, alpha=alpha, beta=beta: beta
    * gamma(alpha, beta, u) ** (i + 2)
    / (i + 2)
    * np.cos(j * beta * u)
    for i in range(-1, n + 1)
    for j in range(0, 3 * (i + 1) + 2 + 1)
    for alpha in alphas
    for beta in betas
]
functions_sin = [
    lambda u, alpha=alpha, beta=beta: beta
    * gamma(alpha, beta, u) ** (i + 2)
    / (i + 2)
    * np.sin(j * beta * u)
    for i in range(-1, n + 1)
    for j in range(0, 3 * (i + 1) + 2 + 1)
    for alpha in alphas
    for beta in betas
]

functions = functions_cos + functions_sin

F = FunctionFamily(
    Interval(0, 1), None, functions
)
print(len(functions))

In [ ]:
interpolation_degree=10
eps_disc = 1e-10
eps_comp = 1e2*eps_disc
eps_quad = 1e-8
disc = Discretizer(precision=eps_disc, min_length=1e-1, interpolation_degree=interpolation_degree, verbose=False)
x_disc, w_disc, endpoints, intervals = disc.adaptive_discretization(F)

In [ ]:
xx = np.linspace(0,1,1000)
plt.plot(xx,functions[100](xx))

In [ ]:
A = construct_A_matrix(x_disc,w_disc,F.functions)
visualise_diagonal_dropoff(A, eps_comp)

In [ ]:
U_disc, rank = compress_sequence_of_functions(F.functions, x_disc, w_disc, eps_comp)

In [ ]:
(x_cheb,),w_cheb = construct_Chebyshev_quadratures((x_disc,),w_disc,U_disc)

In [ ]:
print(x_cheb)

In [ ]:
u_list = interp_legendre(U_disc, endpoints)
r = U_disc.T@w_disc
opt = QuadOptimizer(u_list,r)
x,w = opt.point_reduction(x_cheb, w_cheb, eps_quad)

In [ ]:
f = functions[1]
err = [abs(f(x_disc)@w_disc - f(x)@w) for f in functions]
plt.semilogy(sorted(err))

In [ ]:
print(w)